In [ ]:
import csv
import pandas as pd
import datetime

item = pd.read_csv('LAKEHOUSE_SILVER_LAYER/items1.csv')
prod = pd.read_csv('LAKEHOUSE_SILVER_LAYER/production.csv')
warehouse = pd.read_csv('LAKEHOUSE_SILVER_LAYER/warehouse.csv')

In [ ]:
modwarehouse = warehouse

In [ ]:
modwarehouse["datetime"] = modwarehouse["Registering Date"].apply(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y"))
modwarehouse.columns

Index(['Entry No_', 'Journal Batch Name', 'Line No_', 'Registering Date',
       'Location Code', 'Zone Code', 'Bin Code', 'Description', 'Item No_',
       'Quantity', 'Qty_ (Base)', 'Source Type', 'Source Subtype',
       'Source No_', 'Source Line No_', 'Source Subline No_',
       'Source Document', 'Source Code', 'Reason Code', 'No_ Series',
       'Bin Type Code', 'Cubage', 'Weight', 'Journal Template Name',
       'Whse_ Document No_', 'Whse_ Document Type', 'Whse_ Document Line No_',
       'Entry Type', 'Reference Document', 'Reference No_', 'User ID',
       'Variant Code', 'Qty_ per Unit of Measure', 'Unit of Measure Code',
       'Serial No_', 'Lot No_', 'Warranty Date', 'Expiration Date',
       'Phys Invt Counting Period Code', 'Phys Invt Counting Period Type',
       'Dedicated', 'datetime'],
      dtype='object')

In [ ]:
step1 = modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["datetime"].min().reset_index(name="Min_Registering_Date")
step1["Min_Registering_Date"] = step1["Min_Registering_Date"].apply(lambda x: x.strftime("%d-%m-%Y"))

#step1["Min_Registering_Date"] = modwarehouse[modwarehouse["Lot No_"]]["Registering Date"].min()
#step1.columns = ["Min_Registering_Date"]

#step1 = step1[step1["Quantity"]>0]
#step1

In [ ]:
step1 = step1.set_index(["Lot No_", "Bin Code", "Item No_"])
step1["Quantity"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["Quantity"].sum())

In [ ]:
def func(lis):
    return sorted(lis.unique())[0]
step1["Zone Code"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["Zone Code"].apply(lambda a: sorted(a.unique())[0]))

In [ ]:
#step1.get_group("IDS-21E2-00028", "IGS")#,"104754")
#modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])
step1


Min_Registering_Date  Quantity Zone Code
Lot No_        Bin Code Item No_                                         
IDS-21E2-00028 IGS      1000457            19-05-2021      1000       IGS
                        102078             08-08-2022       175       IGS
                        102565             28-01-2022       350       IGS
                        102671             13-12-2021       200       IGS
                        104754             24-02-2021         0       IGS
...                                               ...       ...       ...
IDS-21E2-01221 IGS      108259             02-12-2021       250       IGS
IDS-21E2-01233 TMG      125948             19-03-2022       175       TMG
IDS-21E2-01236 IGS      147831             16-08-2022       175       IGS
IDS-21E2-01268 TMG      111742             08-04-2022         0       TMG
                        141305             23-02-2021         0       TMG

[4925 rows x 3 columns]

In [ ]:
#step1["Days from Registration"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["datetime"].min().apply(lambda a: datetime.datetime.now() - a).apply(lambda a: a.days))
step1["Days from Registration"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["datetime"].min())
step1["Days from Registration"] = datetime.datetime.now() - step1["Days from Registration"]
step1["Days from Registration"] = step1["Days from Registration"].apply(lambda a: a.days)

In [ ]:
step1 = step1[step1["Quantity"]>0]


In [ ]:
step1

Min_Registering_Date  Quantity Zone Code  \
Lot No_        Bin Code Item No_                                            
IDS-21E2-00028 IGS      1000457            19-05-2021      1000       IGS   
                        102078             08-08-2022       175       IGS   
                        102565             28-01-2022       350       IGS   
                        102671             13-12-2021       200       IGS   
                        105581             15-05-2021      1175       IGS   
...                                               ...       ...       ...   
IDS-21E2-01217 TMG      122402             07-12-2022       200       TMG   
                        123546             24-08-2021       200       TMG   
IDS-21E2-01221 IGS      108259             02-12-2021       250       IGS   
IDS-21E2-01233 TMG      125948             19-03-2022       175       TMG   
IDS-21E2-01236 IGS      147831             16-08-2022       175       IGS   

                                  Days from Registration  
Lot No_        Bin Code Item No_                          
IDS-21E2-00028 IGS      1000457                      602  
                        102078                       156  
                        102565                       348  
                        102671                       394  
                        105581                       606  
...                                                  ...  
IDS-21E2-01217 TMG      122402                        35  
                        123546                       505  
IDS-21E2-01221 IGS      108259                       405  
IDS-21E2-01233 TMG      125948                       298  
IDS-21E2-01236 IGS      147831                       148  

[3994 rows x 4 columns]

In [ ]:
step1.loc["IDS-21E2-00028","IGS","102671"]

Min_Registering_Date      13-12-2021
Quantity                         200
Zone Code                        IGS
Days from Registration           394
Name: (IDS-21E2-00028, IGS, 102671), dtype: object

In [ ]:
step1.to_csv("LAKEHOUSE_GOLD_LAYER/sc_step1.csv")

In [ ]:
print(len(item))
item = item.dropna(subset=['Production BOM No_'])
print(len(item))
item

12000
8049


,No_,No_ 2,Description,Search Description,Description 2,Base Unit of Measure,Price Unit Conversion,Type,Inventory Posting Group,Shelf No_,...,Price_Profit Calculation,Profit _,Costing Method,Unit Cost,Standard Cost,Quoted Price(INR),Quoted Price(FCY),Quoted Currency,Standard Cost_,Production BOM No_
0,135677,NaN,STATIC SHIELDED BAGS OF SIZE 280x220,STATIC SHIELDED BAGS OF SIZE 280x221,NaN,EA,120,0,CN,S001,...,536,50,UC,2541,2554,31,26,USD,86,1000456-SMT
1,119852,NaN,STATIC SHIELDED BAGS OF SIZE 280x220,STATIC SHIELDED BAGS OF SIZE 280x221,NaN,EA,120,0,RM,S001,...,301,50,UC,3799,3799,16,49,USD,86,1000456-SMT
2,147831,NaN,"RoHS,Label, 4X6 inch","RoHS,Label, 4X6 inch",NaN,EA,120,0,PM,S003,...,314,50,UC,3772,3872,50,6,USD,82,1000456-SMT
3,105695,NaN,"RoHSLabel, 3.5 X 1 inch blank (outer box labels/","RoHSLabel, 3.5 X 1 inch blank (outer box labels/",NaN,EA,120,0,PM,S007,...,468,50,UC,2807,2861,29,9,INR,88,1000456-SMT
5,123546,NaN,"RoHS,DRAM-1","RoHS,LABEL-CRUCIAL LABEL--*-54*12.8MM-GLOSS",NaN,EA,120,0,PM,S003,...,337,50,UC,2823,2824,50,5,INR,91,1000457-SMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11993,116912,NaN,"ROHS,Attention Tape","ROHS,Attention Tape",NaN,EA,120,0,CN,S005,...,520,50,UC,3902,3908,54,11,INR,80,1000456-SMT
11995,119852,NaN,"RoHS,LABEL-CRUCIAL LABEL--*-54*12.8MM-GLOSS","RoHS,LABEL-CRUCIAL LABEL--*-54*12.8MM-GLOSS",NaN,EA,120,0,PM,S004,...,660,50,UC,2936,2962,36,5,USD,86,1000456-SMT
11997,112048,NaN,_RARE_,_RARE_,NaN,EA,120,0,CN,S002,...,785,50,UC,2550,2558,27,3,INR,100,1000457-SMT
11998,119852,NaN,OTCT-377*192*309MM-200PCS DIMM-Y,OTCT-377*192*309MM-200PCS DIMM-Y,NaN,EA,120,0,CN,S006,...,453,50,UC,2763,2769,22,37,USD,98,1000457-SMT


In [ ]:
prod

,Production BOM No_,Version Code,Line No_,Type,Description,No_,Unit of Measure Code,Quantity,Position,Position 2,...,Ending Date,Length,Width,Weight,Depth,Calculation Formula,Quantity per,Principal Input,Company,Division
0,1000456-SMT,NaN,11000000,1,STATIC SHIELDED BAGS OF SIZE 280x220,135677,EA,16,SMT,NaN,...,28-07-2026,13,14,17,19,0,65,1,NaN,NaN
1,1000456-SMT,NaN,10277800,1,STATIC SHIELDED BAGS OF SIZE 280x220,119852,EA,66,SMT,NaN,...,14-10-2026,16,16,11,18,0,99,1,NaN,NaN
2,1000456-SMT,NaN,5837686,1,"RoHS,Label, 4X6 inch",147831,EA,99,SMT,NaN,...,19-06-2026,19,13,10,20,0,59,1,NaN,NaN
3,1000456-SMT,NaN,9996220,1,"RoHSLabel, 3.5 X 1 inch blank (outer box labels/",105695,EA,1,SMT,NaN,...,27-04-2026,15,11,12,20,0,66,1,NaN,NaN
4,NaN,NaN,11000000,1,"RoHS,Security Tape",141663,EA,38,SMT,NaN,...,27-03-2026,11,10,20,19,0,72,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1000456-SMT,NaN,4426189,1,"RoHS,LABEL-CRUCIAL LABEL--*-54*12.8MM-GLOSS",119852,EA,1,SMT,NaN,...,10-04-2026,13,12,20,12,0,99,1,NaN,NaN
11996,NaN,NaN,10974911,1,"RoHS PCB BOARD, IQ CONTROLLER",147359,EA,1,SMT,NaN,...,28-07-2026,20,20,19,10,0,29,1,NaN,NaN
11997,1000457-SMT,NaN,10000,1,_RARE_,112048,EA,99,SMT,NaN,...,05-08-2026,18,19,20,10,0,1,1,NaN,NaN
11998,1000457-SMT,NaN,11000000,1,OTCT-377*192*309MM-200PCS DIMM-Y,119852,EA,43,SMT,NaN,...,29-11-2026,14,19,16,19,0,19,1,NaN,NaN


In [ ]:
#step2 = pd.merge(item, prod, on='No_')
#step2 = pd.merge(item,  on='No_', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
#step2.columns.duplicated()
#step2.head()
step2 = pd.concat([item["No_"],prod["No_"]])
step2 = step2.drop_duplicates()
#step2.drop(step2.filter(regex='_DROP$').columns, axis=1, inplace=True)
step2 = pd.DataFrame(step2)
step2 = step2.rename(columns={'No_': 'Item No_'})
step2


,Item No_
0,135677
1,119852
2,147831
3,105695
5,123546
...,...
10058,142861
10553,149879
10620,114559
11610,135872


In [ ]:
step2.to_csv("LAKEHOUSE_GOLD_LAYER/sc_step2.csv")

In [ ]:
step3 = pd.merge(step1, step2, on='Item No_', how="left", suffixes=('', '_DROP'))

step3.drop(step3.filter(regex='_DROP$').columns, axis=1, inplace=True)
print(step3.shape)


(3994, 5)


In [ ]:
step3.to_csv("LAKEHOUSE_GOLD_LAYER/sc_step3.csv")

In [ ]:
step4 = pd.merge(step3, item, left_on='Item No_', right_on='No_')
print(step4.shape)
#step4.drop(step3.filter(regex='_DROP$').columns, axis=1, inplace=True)
step4['Item No_'].equals(step4['No_'])

(655764, 30)


True

In [ ]:
step4.drop(["No_"],axis=1)

,Item No_,Min_Registering_Date,Quantity,Zone Code,Days from Registration,No_ 2,Description,Search Description,Description 2,Base Unit of Measure,...,Price_Profit Calculation,Profit _,Costing Method,Unit Cost,Standard Cost,Quoted Price(INR),Quoted Price(FCY),Quoted Currency,Standard Cost_,Production BOM No_
0,1000457,19-05-2021,1000,IGS,602,NaN,INCT-CB-293*216*90-SHIPPER SODIMM 2-PK TRAY-Y,INCT-CB-293*216*90-SHIPPER SODIMM 2-PK TRAY-Y,NaN,EA,...,583,50,UC,3716,3710,57,17,USD,97,1000457-SMT
1,1000457,19-05-2021,1000,IGS,602,NaN,"RoHS,Label, 5 X 2 inch blank","RoHS,Label, 5 X 2 inch blank",NaN,EA,...,437,50,UC,2663,2639,28,39,USD,88,1000456-SMT
2,1000457,19-05-2021,1000,IGS,602,NaN,"ROHS,Attention Tape","ROHS,Attention Tape",NaN,EA,...,725,50,UC,2521,2926,11,13,USD,98,1000456-SMT
3,1000457,19-05-2021,1000,IGS,602,NaN,"RoHS,DRAM-1","RoHS,DRAM-2",NaN,EA,...,764,50,UC,3192,3470,54,22,USD,91,1000457-SMT
4,1000457,19-05-2021,1000,IGS,602,NaN,"RoHS,Security Tape","RoHS,Security Tape",NaN,EA,...,449,50,UC,3261,3239,55,41,USD,82,1000457-SMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655759,148306,28-11-2022,175,TMG,44,NaN,"RoHS,SCM1G8Z41CD8SA-062E","RoHS,SCM1G8Z41CD8SA-062E",NaN,EA,...,623,50,UC,2961,2967,48,19,USD,77,1000457-SMT
655760,140949,24-07-2021,200,IGS,536,NaN,"RoHS,DRAM-1","RoHS,DRAM-2",NaN,EA,...,511,50,UC,2722,2442,59,31,INR,80,1000456-SMT
655761,140949,24-07-2021,200,IGS,536,NaN,"RoHS,Security Tape","RoHS,Security Tape",NaN,EA,...,323,50,UC,3429,3431,11,29,INR,97,1000457-SMT
655762,140949,24-07-2021,200,IGS,536,NaN,_RARE_,_RARE_,NaN,EA,...,373,50,UC,2983,2936,35,8,INR,81,1000457-SMT
